### 

In [1]:
import time
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer

# ================= CẤU HÌNH =================
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

# Model embedding tiếng Việt tốt nhất hiện nay
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"
# Kích thước vector của model này là 768
VECTOR_DIMENSION = 768 

# Tên các Index vector sẽ tạo (cho 2 loại label chính trong code của ông)
INDEXES = {
    "searchable_index": "Searchable",
    "general_info_index": "GeneralInfo"
}

def create_vector_indexes(driver):
    """
    Tạo Vector Index trong Neo4j để phục vụ tìm kiếm RAG.
    Nếu không có cái này, ông không thể dùng hàm vector similarity search được.
    """
    print("⚙️ Đang kiểm tra và tạo Vector Index...")
    with driver.session() as session:
        for index_name, label in INDEXES.items():
            # Câu lệnh Cypher tạo index cho Neo4j 5.x trở lên
            query = f"""
            CREATE VECTOR INDEX `{index_name}` IF NOT EXISTS
            FOR (n:{label}) ON (n.embedding)
            OPTIONS {{
                indexConfig: {{
                    `vector.dimensions`: {VECTOR_DIMENSION},
                    `vector.similarity_function`: 'cosine'
                }}
            }}
            """
            try:
                session.run(query)
                print(f"   ✅ Đã tạo/kiểm tra index: {index_name} cho Label: {label}")
            except Exception as e:
                print(f"   ⚠️ Lỗi tạo index {index_name}: {e}")

def fetch_nodes_without_embedding(tx, batch_size=200):
    """
    Lấy các node có 'embedding_text' nhưng chưa có 'embedding' (vector)
    """
    query = """
    MATCH (n)
    WHERE n.embedding_text IS NOT NULL 
      AND n.embedding IS NULL
      AND (n:Searchable OR n:GeneralInfo)
    RETURN elementId(n) AS id, n.embedding_text AS text
    LIMIT $limit
    """
    result = tx.run(query, limit=batch_size)
    return [{"id": record["id"], "text": record["text"]} for record in result]

def update_node_embeddings(tx, updates):
    """
    Update vector ngược lại vào Neo4j
    """
    query = """
    UNWIND $updates AS row
    MATCH (n) WHERE elementId(n) = row.id
    
    // Sửa lỗi: Thêm YIELD node để tuân thủ cú pháp Neo4j 5.x
    CALL db.create.setVectorProperty(n, 'embedding', row.vector)
    YIELD node 
    
    // Kết thúc bằng RETURN để transaction hoàn tất hợp lệ
    RETURN count(node)
    """
    tx.run(query, updates=updates)
    
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    
    # 1. Load Model (Load 1 lần dùng mãi mãi)
    print(f"🚀 Đang tải model AI: {MODEL_NAME}...")
    model = SentenceTransformer(MODEL_NAME)
    print("✅ Model đã sẵn sàng!")

    # 2. Tạo Index trước (Quan trọng)
    create_vector_indexes(driver)

    # 3. Vòng lặp xử lý (Batch processing)
    batch_size = 200
    total_processed = 0
    
    print("⏳ Bắt đầu quá trình Vector hóa dữ liệu trong Neo4j...")
    
    with driver.session() as session:
        while True:
            # A. Lấy dữ liệu chưa có vector
            nodes = session.execute_read(fetch_nodes_without_embedding, batch_size)
            
            if not nodes:
                print("🎉 Đã xử lý hết toàn bộ dữ liệu! Không còn node nào thiếu vector.")
                break
            
            print(f"   🔹 Đang xử lý batch {len(nodes)} node...")
            
            # B. Tính toán Vector (Embedding)
            texts = [n["text"] for n in nodes]
            # encode trả về numpy array, cần convert sang list để nạp vào Neo4j
            embeddings = model.encode(texts, show_progress_bar=False).tolist()
            
            # C. Chuẩn bị dữ liệu update
            updates = []
            for i, node in enumerate(nodes):
                updates.append({
                    "id": node["id"],
                    "vector": embeddings[i]
                })
            
            # D. Ghi xuống DB
            session.execute_write(update_node_embeddings, updates)
            
            total_processed += len(nodes)
            print(f"   ✅ Đã update xong {total_processed} node.")

    driver.close()

if __name__ == "__main__":
    main()

d:\RAG\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Đang tải model AI: bkai-foundation-models/vietnamese-bi-encoder...
✅ Model đã sẵn sàng!
⚙️ Đang kiểm tra và tạo Vector Index...
   ✅ Đã tạo/kiểm tra index: searchable_index cho Label: Searchable
   ✅ Đã tạo/kiểm tra index: general_info_index cho Label: GeneralInfo
⏳ Bắt đầu quá trình Vector hóa dữ liệu trong Neo4j...
   🔹 Đang xử lý batch 200 node...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. db.create.setVectorProperty is deprecated. It is replaced by db.create.setNodeVectorProperty.', position=<SummaryInputPosition line=6, column=5, offset=137>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 137, 'line': 6, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    UNWIND $updates AS row\n    MATCH (n) WHERE elementId(n) = row.id\n\n    // Sửa lỗi: Thêm YIELD node để tuân thủ cú pháp Neo4j 5.x\n    CALL db.create.setVectorProperty(n, 'embedding', row.vector)\n    YIELD node \n\n    // Kết thúc bằng RETURN để transaction hoàn tất hợp lệ\n    RETURN count(node)\n    "


   ✅ Đã update xong 200 node.
   🔹 Đang xử lý batch 200 node...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. db.create.setVectorProperty is deprecated. It is replaced by db.create.setNodeVectorProperty.', position=<SummaryInputPosition line=6, column=5, offset=137>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 137, 'line': 6, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    UNWIND $updates AS row\n    MATCH (n) WHERE elementId(n) = row.id\n\n    // Sửa lỗi: Thêm YIELD node để tuân thủ cú pháp Neo4j 5.x\n    CALL db.create.setVectorProperty(n, 'embedding', row.vector)\n    YIELD node \n\n    // Kết thúc bằng RETURN để transaction hoàn tất hợp lệ\n    RETURN count(node)\n    "


...

   ✅ Đã update xong 1200 node.
   🔹 Đang xử lý batch 200 node...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. db.create.setVectorProperty is deprecated. It is replaced by db.create.setNodeVectorProperty.', position=<SummaryInputPosition line=6, column=5, offset=137>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 137, 'line': 6, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    UNWIND $updates AS row\n    MATCH (n) WHERE elementId(n) = row.id\n\n    // Sửa lỗi: Thêm YIELD node để tuân thủ cú pháp Neo4j 5.x\n    CALL db.create.setVectorProperty(n, 'embedding', row.vector)\n    YIELD node \n\n    // Kết thúc bằng RETURN để transaction hoàn tất hợp lệ\n    RETURN count(node)\n    "


   ✅ Đã update xong 1400 node.
   🔹 Đang xử lý batch 78 node...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. db.create.setVectorProperty is deprecated. It is replaced by db.create.setNodeVectorProperty.', position=<SummaryInputPosition line=6, column=5, offset=137>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 137, 'line': 6, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    UNWIND $updates AS row\n    MATCH (n) WHERE elementId(n) = row.id\n\n    // Sửa lỗi: Thêm YIELD node để tuân thủ cú pháp Neo4j 5.x\n    CALL db.create.setVectorProperty(n, 'embedding', row.vector)\n    YIELD node \n\n    // Kết thúc bằng RETURN để transaction hoàn tất hợp lệ\n    RETURN count(node)\n    "


   ✅ Đã update xong 1478 node.
🎉 Đã xử lý hết toàn bộ dữ liệu! Không còn node nào thiếu vector.


### Phương pháp đang sử dụng là Dense Vector Embedding (Nhúng vector mật độ cao) sử dụng kiến trúc Bi-Encoder.
Cụ thể về kỹ thuật:
#### 1. Phương pháp kỹ thuật: Bi-Encoder Dense RetrievalBi-Encoder là gì? 
Model bkai-foundation-models/vietnamese-bi-encoder hoạt động theo cơ chế Bi-Encoder. Nghĩa là nó "đọc" đoạn văn bản (input text) một cách độc lập và nén toàn bộ ý nghĩa của đoạn văn đó thành một chuỗi số cố định (vector 768 chiều).Cơ chế lưu trữ: Đoạn code trên thực hiện việc Indexing (Lập chỉ mục). Nó biến đổi dữ liệu thô (Text) -> Vector -> Lưu vào Neo4j.Thuật toán so khớp: Khi tạo index (vector.similarity_function: 'cosine'), bạn đang quy định rằng sau này việc tìm kiếm sẽ dùng Cosine Similarity (độ tương đồng cosin - góc giữa 2 vector) để đo độ giống nhau về ngữ nghĩa.
#### 2. Code này CÓ gây ra "Ảo giác ngữ nghĩa" không?
Câu trả lời là: CÓ, bản thân cái vector được tạo ra từ code này chứa đựng nguy cơ đó.Tại sao?
=> Nén thông tin (Information Compression): Model BKAI nén tất cả mọi thứ: Địa danh (Hà Nội) + Đối tượng (Chùa) + Tính chất (Cổ kính) vào chung 1 vector duy nhất.
Trọng số không kiểm soát: Trong quá trình train, model có thể "học" rằng từ "Chùa" quan trọng hơn từ "Hà Nội".Khi đó: Vector của "Chùa Một Cột (Hà Nội)" sẽ nằm rất gần Vector của "Chùa Thiên Mụ (Huế)" (vì cùng là Chùa).Nó nằm xa Vector của "Phở Bát Đàn (Hà Nội)" (dù cùng địa lý nhưng khác hẳn về nội dung).
#### -> Hậu quả: Nếu bạn chỉ dùng cái vector này để search (Vector Search thuần túy), máy sẽ ưu tiên trả về "Các ngôi chùa ở khắp nơi" thay vì "Các địa điểm tại Hà Nội". Đây chính là nguyên nhân gốc rễ của ảo giác địa lý.

#### => Đây là bước bắt buộc phải làm.Bạn không thể sửa "vector" để nó hết ảo giác (vì bản chất vector là xác suất và ngữ nghĩa mờ). Phải sửa ở cách truy vấn (Query Strategy). 
Đây là bước chuẩn bị nguyên liệu (Data + Vector). 
Phải áp dụng phương pháp Hybrid Search (Tìm kiếm lai) ở bước Viết API tìm kiếm (bước sau):Dùng Graph (Neo4j) để khóa địa lý: Dùng câu lệnh Cypher MATCH (n)-[:LOCATED_IN]->(p:Province {name: 'Hà Nội'}) để khoanh vùng.Dùng Vector để tìm ngữ nghĩa: Chỉ so sánh vector trong cái vùng đã khoanh đó.